# Getting Started with Pilot-Streaming on AWS

In the first step we need to import all required packages and modules into the Python Path

The Pilot-Compute Description is a simple key/value style description of the cluster environment that should be started. 

In [ ]:
# System Libraries
import sys, os
sys.path.append("..")
import pandas as pd

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
 
import pyspark
import os
import boto3
boto3.setup_default_session(profile_name='dev')
    
# Pilot-Streaming
import pilot.streaming
sys.modules['pilot.streaming']

# 1. Kinesis

In [ ]:
pilot_compute_description = {
    "resource":"kinesis://awscloud.com",
    "number_cores": 1,
    "type":"kinesis"
}

Start Spark Cluster and Wait for Startup Completion

In [ ]:
%%time
kinesis_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
kinesis_pilot.wait()

In [ ]:
kinesis_pilot.get_details()

In [ ]:
kinesis_pilot.get_id()

In [ ]:
kinesis_client = boto3.client('kinesis', region_name='us-east-1')
put_response = kinesis_client.put_record(
                        StreamName=kinesis_pilot.get_id().split("/")[1],
                        Data="Hello World",
                        PartitionKey="A")

# 2. Lambda

In [ ]:
def lambda_handler(event, context):
    import base64
    print("Lambda Function called")
    for record in event['Records']:
        #Kinesis data is base64 encoded so decode here
        payload=base64.b64decode(record["kinesis"]["data"])
        print("Decoded payload: " + str(payload))

In [ ]:
pilot_compute_description = {
    "resource":"kinesis://awscloud.com",
    "number_cores": 1,
    "lambda_input_data": kinesis_pilot.get_id(),
    "lambda_function": lambda_handler,
    "type":"lambda"
}

In [ ]:
%%time
lambda_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
lambda_pilot.wait()

In [ ]:
lambda_pilot.get_details()

In [ ]:
def lambda_handler(event, context):
    import base64
    print("Lambda Function called Version 2")
    for record in event['Records']:
        #Kinesis data is base64 encoded so decode here
        payload=base64.b64decode(record["kinesis"]["data"])
        print("Decoded payload: " + str(payload))
        
lambda_pilot.submit(lambda_handler) #update function inside of Lambda

In [ ]:
kinesis_pilot.cancel()
lambda_pilot.cancel()

# 3. EC2

In [ ]:
import boto3
import json
boto3.setup_default_session(profile_name='dev')
ec2_client = boto3.resource('ec2', region_name='us-east-1')

#{
#    "resource":"ec2://awscloud.com",
#    "number_cores" : 1,
#    "ec2_ssh_username":"ubuntu",
#    "ec2_ssh_keyname":"xxx",
#    "ec2_ssh_keyfile":"",    
#    "ec2_vpc_id" : "xxx",   
#    "ec2_subnet_id": "xxx",
#    "ec2_image_id" : "xxx", 
#    "ec2_instance_type" : "t2.nano",
#    "type":"ec2"
#}

ec2_pilot_description = json.load(open("aws_pilotcomputedescription.json", "r"))

In [ ]:
ec2_client.create_instances(ImageId=ec2_pilot_description["ec2_image_id"],
                            InstanceType=ec2_pilot_description["ec2_instance_type"],
                            KeyName=ec2_pilot_description["ec2_ssh_keyname"],
                            SubnetId=ec2_pilot_description["ec2_subnet_id"],
                            Placement={'AvailabilityZone': 'us-east-1a'},
                            MinCount=1, MaxCount=1)

In [ ]:
sec_group.authorize_ingress(CidrIp='0.0.0.0/0',
                        IpProtocol='tcp',
                        FromPort=22,
                        ToPort=22)

In [ ]:
# Boto 3
for status in ec2_client.meta.client.describe_instance_status()['InstanceStatuses']:
    print(status)